# Pytorch Notebook

In [1]:
import torch
import torchvision
import numpy

### Define Fully Connected Feed Forward NN $f_{\theta}$ 

### Calculate Loss $L(\theta)$

### Calculate $\nabla_\theta L(\theta, x)|_{x=x_0}$

### Calculate $\nabla_x L(\theta, x)|_{x=x_0}$

### Update $\theta \leftarrow \theta + \alpha \nabla_\theta L(\theta, x)|_{x=x_0}$

### Update weight by minibatch $\frac{1}{N} \sum_{n=1}^N \nabla_\theta L(\theta, x)|_{x=x_n}$

### Update $x \leftarrow x + \alpha \nabla_x L(\theta, x)|_x$

### Copy $\theta_1$ to $\theta_2$

### Copy $\theta_1$ to $\theta_2$ and update only $\theta_1$ by $\theta \leftarrow \theta + \alpha \nabla_\theta L(\theta, x)|_{x=x_0}$

### Stochastic sampling from $f_\theta(x)$

### Inject noise (reparametrization trick) and cal grad $\nabla_\theta f_\theta(x, \epsilon)$, $\epsilon\sim {\cal N}(\mu, \sigma^2)$

### Use weighted update $\theta \leftarrow \theta + \alpha \delta \nabla_\theta L(\theta, x)|_{x=x_0}$, $\delta \sim {\cal N}(0, 1)$

### Use noisy weight $f(x, \theta + \epsilon)$, $\epsilon \sim {\cal N}(1, 0)$

### Update $\theta$ by the approximated variational method

# Convolution Neural Networks

### Define CNN $f_\theta(x)$

### Update weight by minibatch for CNN

### Update $x \leftarrow x + \alpha \nabla_x L(\theta, x)|_x$ for CNN

# Recurrent Neural Networks

### Define RNN $f_\theta(x_0, x_1, \dots, x_T)$

### Update weight by minibatch for RNN

### Update $x \leftarrow x + \alpha \nabla_x L(\theta, x)|_x$ for RNN